# Layer Analytics of Neural Networks


In [74]:
import os, sys, math, datetime
import psutil
#import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image

# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model
from mltk.utils.commander import query_platform

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



In [89]:
# Configure pandas to show all columns
pd.set_option('display.max_columns', None)

In [9]:
models_dir = Path.cwd().joinpath("models")
if not models_dir.exists():
    print(f"{models_dir} does not exist.")
    models_dir.mkdir()
    print(f"Created path: {models_dir}.")


In [10]:
input_shape =(96,96,3)
classes = 2
alpha = 0.25

# generate folder structure for saving the models
channels = input_shape[-1]
print(channels)


3


# Helper Functions

In [77]:
# Helper functions

def flat_size(shape : list):
    out = 1
    for k in shape:
        out *= k
        
    return out

# Create the model

In [11]:
mobilenet = tf.keras.applications.mobilenet.MobileNet(
    input_shape=input_shape,
    alpha=alpha,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None, #'imagenet'
    input_tensor=None,
    pooling=None,
    classes=classes,
    classifier_activation='softmax',
    #**kwargs
)

In [12]:
model = mobilenet
model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

In [13]:
summary = model.summary(expand_nested=True)

Model: "mobilenet_0.25_96"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 48, 48, 8)         216       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 48, 48, 8)        32        
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 48, 48, 8)         0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 8)        72        
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 48, 48, 8)        32        
 ation)                                          

In [73]:
mltk_summary = summarize_model(model)
print(mltk_summary)
type(mltk_summary)

Model: "mobilenet_0.25_96"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 48, 48, 8)         216       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 48, 48, 8)        32        
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 48, 48, 8)         0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 8)        72        
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 48, 48, 8)        32        
 ation)                                          

str

In [76]:
# MLTK profile model reads the mode from a path
profiling_results = profile_model(model, accelerator='MVP', build=True)

RuntimeError: Must provide MltkModel instance, name of MltkModel, other .py path to model specification to use the build option

In [70]:
# Show model in local version of Netron.app
view_model(model, tflite= True, build=True)

Serving 'C:/Users/Susanne/AppData/Local/Temp/Susanne/mltk/tmp_models/model.h5' at http://localhost:8080
Stopping http://localhost:8080


In [14]:
model_name = model.name
model_name = f"{model_name}_c{channels}"
print(model_name)


mobilenet_0.25_96_c3


In [15]:
models_path = models_dir.joinpath(model_name)
if not models_path.exists():
    print(f"{models_path} does not exist.")
    models_path.mkdir()
    print(f"Created path: {models_path}.")

models_summary_path = models_dir.joinpath(model_name, f"{model_name}.txt")
models_image_path = models_dir.joinpath(model_name, f"{model_name}.png")
print(models_summary_path)
models_tflite_path = models_dir.joinpath(model_name, f"{model_name}.tflite")
models_tflite_opt_path = models_dir.joinpath(model_name, f"{model_name}_INT8.tflite")
print(models_tflite_path)

i:\tinyml\tiny_cnn\models\mobilenet_0.25_96_c3\mobilenet_0.25_96_c3.txt
i:\tinyml\tiny_cnn\models\mobilenet_0.25_96_c3\mobilenet_0.25_96_c3.tflite


In [16]:
# Get the details from the different layers of the model

In [17]:
layers =[layer for layer in model.layers]
print(layers)

[<keras.engine.input_layer.InputLayer object at 0x00000293AE04C5E0>, <keras.layers.convolutional.conv2d.Conv2D object at 0x00000293AE107DF0>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002938BA77A90>, <keras.layers.activation.relu.ReLU object at 0x00000293AE08C1C0>, <keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x00000293AE0A4D90>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002938BA66E00>, <keras.layers.activation.relu.ReLU object at 0x00000293AE0F1600>, <keras.layers.convolutional.conv2d.Conv2D object at 0x00000293ADFDBE50>, <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x000002938BABEBF0>, <keras.layers.activation.relu.ReLU object at 0x00000293ADF22BF0>, <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x00000293A7FF8520>, <keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D object at 0x00000293A7FF6E00>, <keras.layers.normalizati

In [18]:
# tf.keras.utils.plot_model(model, 
#     to_file=models_image_path,
#     show_shapes=True, 
#     show_dtype=False,
#     show_layer_names=True, 
#     rankdir='TB', # TB for vertical plot, LR for horizontal plot
#     expand_nested=True, 
#     layer_range=None,
#     dpi=200,
#     show_layer_activations= True)

# Layer Metrics Helper Functions

In [81]:
def Conv2D_Metrics(layer):
    """
    Code adapted from https://github.com/SiliconLabs/mltk/blob/master/mltk/core/model/metrics/layers/conv.py
    """

    if isinstance(layer, keras.layers.Conv2D):
        _, _, _, in_depth = layer.input_shape # layer.input_data.shape
        _, out_h, out_w, out_depth = layer.output_shape # layer.output_data.shape
    else:
        if layer.data_format == "channels_first":
            _, in_depth, _, _ = layer.input_shape
            _, out_depth, out_h, out_w, = layer.output_shape
        elif layer.data_format == "channels_last":
            _, _, _, in_depth = layer.input_shape
            _, out_h, out_w, out_depth = layer.output_shape

    w_h, w_w =  layer.kernel_size
    name = layer.name
    macs = w_h * w_w * in_depth * out_w * out_h * out_depth
    ops = macs * 2 

    if  layer.use_bias:
        ops += out_w * out_h * out_depth
    
    return (name, macs, ops)

In [19]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.input.shape , layer.output_shape, layer.outbound_nodes, layer.compute_dtype, layer.count_params() )#layer.get_config())
    if isinstance(layer, keras.layers.InputLayer):
        print(f"Input Layer: {type(layer)}")

0 input_2 (None, 96, 96, 3) [(None, 96, 96, 3)] [<keras.engine.node.Node object at 0x000002938BA77A60>] float32 0
Input Layer: <class 'keras.engine.input_layer.InputLayer'>
1 conv1 (None, 96, 96, 3) (None, 48, 48, 8) [<keras.engine.node.Node object at 0x0000029384E77AF0>] float32 216
2 conv1_bn (None, 48, 48, 8) (None, 48, 48, 8) [<keras.engine.node.Node object at 0x00000293AE0A5360>] float32 32
3 conv1_relu (None, 48, 48, 8) (None, 48, 48, 8) [<keras.engine.node.Node object at 0x00000293ADF22680>] float32 0
4 conv_dw_1 (None, 48, 48, 8) (None, 48, 48, 8) [<keras.engine.node.Node object at 0x000002938BABD210>] float32 72
5 conv_dw_1_bn (None, 48, 48, 8) (None, 48, 48, 8) [<keras.engine.node.Node object at 0x00000293ADFDAB00>] float32 32
6 conv_dw_1_relu (None, 48, 48, 8) (None, 48, 48, 8) [<keras.engine.node.Node object at 0x000002938BABF220>] float32 0
7 conv_pw_1 (None, 48, 48, 8) (None, 48, 48, 16) [<keras.engine.node.Node object at 0x00000293A7FF8370>] float32 128
8 conv_pw_1_bn (N

In [49]:
def print_layer_config(layer, layer_type):
    if isinstance(layer, layer_type):
        print(f"  Layer: {type(layer)}")
        print(layer.get_config())
        print()
    else:
        pass
    return layer.layer_config

In [90]:

layer_list = []
for i, layer in enumerate(model.layers):
    layer_stats = {}
    layer_stats["index"] = i
    layer_stats["name"] = layer.name
    layer_stats["layer_type"] = type(layer) # TODO: find a better representation
    layer_stats["input_shape"] = layer.input_shape
    layer_stats["output_shape"] = layer.output_shape
    layer_stats["dtype"] = layer.compute_dtype
    layer_stats["params"] = layer.count_params()
    layer_stats["variables"] = len(layer.variables)
    try:
        layer_stats["weights_shape"] = layer.weights[0].shape
        layer_stats["weights_name"] = layer.weights[0].name
        layer_stats["weights_dtype"] = layer.weights[0].dtype
    except:
        layer_stats["weights_shape"] = 0
        layer_stats["weights_name"] = 0
        layer_stats["weights_dtype"] = 0    

    layer_config = layer.get_config()

    # Set default values for parameters that are not present in each layer
    layer_stats["OPS"] = 0
    layer_stats["MACS"] = 0
    layer_stats["filters"]= 0
    layer_stats["trainable"] = 0
    layer_stats["kernel_size"] = 0
    layer_stats["strides"] = 0
    layer_stats["padding"] = 0
    layer_stats["dilation_rate"] = 0
    layer_stats["groups"] = 0
    layer_stats["activation"] = 0
    layer_stats["use_bias"] = 0
    layer_stats["data_format"] = 0

    # if isinstance(layer, keras.layers.InputLayer):
    #     print(f"  Layer: {type(layer)}")
    #     print(layer_config)
    #     print()

    if isinstance(layer, keras.layers.Conv2D ):
        # print(f"  Layer: {type(layer)}")
        # print(layer_config)
        # print()
        name, macs, ops = Conv2D_Metrics(layer)

        layer_stats["OPS"] = ops
        layer_stats["MACS"] = macs
        layer_stats["filters"]= layer_config["filters"]
        layer_stats["trainable"] = layer_config["trainable"]
        layer_stats["kernel_size"] = layer_config["kernel_size"]
        layer_stats["strides"] = layer_config["strides"]
        layer_stats["padding"] = layer_config["padding"]
        layer_stats["dilation_rate"] = layer_config["dilation_rate"]
        layer_stats["groups"] = layer_config["groups"]
        layer_stats["activation"] = layer_config["activation"]
        layer_stats["use_bias"] = layer_config["use_bias"]
        layer_stats["data_format"] = layer_config["data_format"]


    if isinstance(layer, keras.layers.DepthwiseConv2D ):
        print(f"  Layer: {type(layer)}")
        print(layer_config)
        print()

        name, macs, ops = Conv2D_Metrics(layer)   # TODO: Double check if this is the right calculation

        #layer_stats["filters"]= layer_config["filters"]  # TODO: key error -> refactor with Conv2D
        layer_stats["OPS"] = ops
        layer_stats["MACS"] = macs
        layer_stats["trainable"] = layer_config["trainable"]
        layer_stats["kernel_size"] = layer_config["kernel_size"]
        layer_stats["strides"] = layer_config["strides"]
        layer_stats["padding"] = layer_config["padding"]
        layer_stats["dilation_rate"] = layer_config["dilation_rate"]
        layer_stats["groups"] = layer_config["groups"]
        layer_stats["activation"] = layer_config["activation"]
        layer_stats["use_bias"] = layer_config["use_bias"]
        layer_stats["data_format"] = layer_config["data_format"]


    # if isinstance(layer, keras.layers.Dense):
    #     print(f"  Layer: {type(layer)}")
    #     print(layer_config)
    #     print()

    # if isinstance(layer, keras.layers.BatchNormalization ):
    #     print(f"  Layer: {type(layer)}")
    #     print(layer_config)
    #     print()
    layer_list.append(layer_stats)

layer_stats_df = pd.DataFrame.from_dict(layer_list)
layer_stats_df

  Layer: <class 'keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D'>
{'name': 'conv_dw_1', 'trainable': True, 'dtype': 'float32', 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}

  Layer: <class 'keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D'>
{'name': 'conv_dw_2', 'trainable': True, 'dtype': 'float32', 'kernel_size': (3, 3), 'strides': (2, 2), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bi

,index,name,layer_type,input_shape,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format
0,0,input_2,<class 'keras.engine.input_layer.InputLayer'>,"[(None, 96, 96, 3)]","[(None, 96, 96, 3)]",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,conv1,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 96, 96, 3)","(None, 48, 48, 8)",float32,216,1,"(3, 3, 3, 8)",conv1/kernel:0,<dtype: 'float32'>,995328,497664,8,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,False,channels_last
2,2,conv1_bn,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 8)","(None, 48, 48, 8)",float32,32,4,(8),conv1_bn/gamma:0,<dtype: 'float32'>,0,0,0,0,0,0,0,0,0,0,0,0
3,3,conv1_relu,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 8)","(None, 48, 48, 8)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,conv_dw_1,<class 'keras.layers.convolutional.depthwise_c...,"(None, 48, 48, 8)","(None, 48, 48, 8)",float32,72,1,"(3, 3, 8, 1)",conv_dw_1/depthwise_kernel:0,<dtype: 'float32'>,2654208,1327104,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,False,channels_last
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,global_average_pooling2d_1,<class 'keras.layers.pooling.global_average_po...,"(None, 3, 3, 256)","(None, 1, 1, 256)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
87,87,dropout,<class 'keras.layers.regularization.dropout.Dr...,"(None, 1, 1, 256)","(None, 1, 1, 256)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
88,88,conv_preds,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 1, 1, 256)","(None, 1, 1, 2)",float32,514,2,"(1, 1, 256, 2)",conv_preds/kernel:0,<dtype: 'float32'>,1026,512,2,True,"(1, 1)","(1, 1)",same,"(1, 1)",1,linear,True,channels_last
89,89,reshape_2,<class 'keras.layers.reshaping.reshape.Reshape'>,"(None, 1, 1, 2)","(None, 2)",float32,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
layer_stats_df["layer_type"].unique()

array([<class 'keras.engine.input_layer.InputLayer'>,
       <class 'keras.layers.convolutional.conv2d.Conv2D'>,
       <class 'keras.layers.normalization.batch_normalization.BatchNormalization'>,
       <class 'keras.layers.activation.relu.ReLU'>,
       <class 'keras.layers.convolutional.depthwise_conv2d.DepthwiseConv2D'>,
       <class 'keras.layers.reshaping.zero_padding2d.ZeroPadding2D'>,
       <class 'keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D'>,
       <class 'keras.layers.regularization.dropout.Dropout'>,
       <class 'keras.layers.reshaping.reshape.Reshape'>,
       <class 'keras.layers.core.activation.Activation'>], dtype=object)

In [82]:
name, macs, ops = Conv2D_Metrics(model.layers[1])
print(name, macs, ops)

conv1 497664 995328


In [21]:
model.layers[1].activation

<function keras.activations.linear(x)>

In [22]:
model.layers[1].get_config()

{'name': 'conv1',
 'trainable': True,
 'dtype': 'float32',
 'filters': 8,
 'kernel_size': (3, 3),
 'strides': (2, 2),
 'padding': 'same',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'groups': 1,
 'activation': 'linear',
 'use_bias': False,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [61]:
print(model.layers[1].weights[0].shape)
print(model.layers[1].weights[0].name)

(3, 3, 3, 8)
conv1/kernel:0


In [26]:
for layer in model.layers:
    print(layer.name, layer.compute_dtype ,len(layer.variables) , layer.input_shape, layer.output_shape)

input_2 float32 0 [(None, 96, 96, 3)] [(None, 96, 96, 3)]
conv1 float32 1 (None, 96, 96, 3) (None, 48, 48, 8)
conv1_bn float32 4 (None, 48, 48, 8) (None, 48, 48, 8)
conv1_relu float32 0 (None, 48, 48, 8) (None, 48, 48, 8)
conv_dw_1 float32 1 (None, 48, 48, 8) (None, 48, 48, 8)
conv_dw_1_bn float32 4 (None, 48, 48, 8) (None, 48, 48, 8)
conv_dw_1_relu float32 0 (None, 48, 48, 8) (None, 48, 48, 8)
conv_pw_1 float32 1 (None, 48, 48, 8) (None, 48, 48, 16)
conv_pw_1_bn float32 4 (None, 48, 48, 16) (None, 48, 48, 16)
conv_pw_1_relu float32 0 (None, 48, 48, 16) (None, 48, 48, 16)
conv_pad_2 float32 0 (None, 48, 48, 16) (None, 49, 49, 16)
conv_dw_2 float32 1 (None, 49, 49, 16) (None, 24, 24, 16)
conv_dw_2_bn float32 4 (None, 24, 24, 16) (None, 24, 24, 16)
conv_dw_2_relu float32 0 (None, 24, 24, 16) (None, 24, 24, 16)
conv_pw_2 float32 1 (None, 24, 24, 16) (None, 24, 24, 32)
conv_pw_2_bn float32 4 (None, 24, 24, 32) (None, 24, 24, 32)
conv_pw_2_relu float32 0 (None, 24, 24, 32) (None, 24, 24, 32